# Chapter 10

# Data Aggregation and Group Operations

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

We will learn:
- Split a pandas object into pieces using one or more keys
- Calculate group summary statistics
- Apply within-group transformations or other manipulations
- Compute pivot tables and cross-tabulations
- Perform quantile analysis and other statistical group analyses

---

# 10.1 GroupBy Mechanics

split-apply-combine

Grouping key form:
- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

In [4]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [5]:
df

,key1,key2,data1,data2
0,a,one,1.483659,0.242712
1,a,two,0.091543,0.406203
2,b,one,2.543224,-0.714365
3,b,two,2.233232,0.249677
4,a,one,0.966602,0.025725


In [12]:
# the first way (compute the mean)
grouped = df['data1'].groupby(df['key1'])

In [13]:
grouped 
# the intermediate data

The idea is
that this object has all of the information needed to then apply some operation to
each of the groups.

In [14]:
grouped.mean()

key1
a    0.847268
b    2.388228
Name: data1, dtype: float64

In [17]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.847268,0.703688,0.091543,0.529072,0.966602,1.225130,1.483659
b,2.0,2.388228,0.219197,2.233232,2.310730,2.388228,2.465726,2.543224


In [19]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [20]:
means

key1  key2
a     one     1.225130
      two     0.091543
b     one     2.543224
      two     2.233232
Name: data1, dtype: float64

In [21]:
means.unstack()

key2,one,two
key1,,
a,1.225130,0.091543
b,2.543224,2.233232


In [22]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [23]:
df['data1'].groupby([states, years]).mean()
# the keys are Series with the right length

California  2005    0.091543
            2006    2.543224
Ohio        2005    1.858445
            2006    0.966602
Name: data1, dtype: float64

In [24]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.847268,0.224880
b,2.388228,-0.232344


In [26]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.225130  0.134218
     two   0.091543  0.406203
b    one   2.543224 -0.714365
     two   2.233232  0.249677

In [28]:
df.groupby(['key1', 'key2']).size()
# count the number of occurences

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

---

In [29]:
df['key1'].groupby(df['key2']).size()

key2
one    3
two    2
Name: key1, dtype: int64

---

## Iterating Over Groups

In [35]:
for name, group in df.groupby('key1'):
    print(name)
    print(type(group))
    print(group)

a
<class 'pandas.core.frame.DataFrame'>
  key1 key2     data1     data2
0    a  one  1.483659  0.242712
1    a  two  0.091543  0.406203
4    a  one  0.966602  0.025725
b
<class 'pandas.core.frame.DataFrame'>
  key1 key2     data1     data2
2    b  one  2.543224 -0.714365
3    b  two  2.233232  0.249677


In [41]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print("key1: %s, key2: %s" % (k1, k2))
    print(group); print()

key1: a, key2: one
  key1 key2     data1     data2
0    a  one  1.483659  0.242712
4    a  one  0.966602  0.025725

key1: a, key2: two
  key1 key2     data1     data2
1    a  two  0.091543  0.406203

key1: b, key2: one
  key1 key2     data1     data2
2    b  one  2.543224 -0.714365

key1: b, key2: two
  key1 key2     data1     data2
3    b  two  2.233232  0.249677



Computing a dict of the data pieces as a one-liner:

In [42]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one  1.483659  0.242712
  1    a  two  0.091543  0.406203
  4    a  one  0.966602  0.025725), ('b',   key1 key2     data1     data2
  2    b  one  2.543224 -0.714365
  3    b  two  2.233232  0.249677)]

In [45]:
list(df.groupby(['key1', 'key2']))

[(('a', 'one'),   key1 key2     data1     data2
  0    a  one  1.483659  0.242712
  4    a  one  0.966602  0.025725),
 (('a', 'two'),   key1 key2     data1     data2
  1    a  two  0.091543  0.406203),
 (('b', 'one'),   key1 key2     data1     data2
  2    b  one  2.543224 -0.714365),
 (('b', 'two'),   key1 key2     data1     data2
  3    b  two  2.233232  0.249677)]

In [46]:
len(list(df.groupby(['key1', 'key2'])))

4

In [47]:
list(df.groupby(['key1', 'key2']))[0]

(('a', 'one'),   key1 key2     data1     data2
 0    a  one  1.483659  0.242712
 4    a  one  0.966602  0.025725)

In [50]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one  1.483659  0.242712
  1    a  two  0.091543  0.406203
  4    a  one  0.966602  0.025725), ('b',   key1 key2     data1     data2
  2    b  one  2.543224 -0.714365
  3    b  two  2.233232  0.249677)]

In [48]:
pieces = dict(list(df.groupby('key1')))

In [49]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.483659  0.242712
 1    a  two  0.091543  0.406203
 4    a  one  0.966602  0.025725, 'b':   key1 key2     data1     data2
 2    b  one  2.543224 -0.714365
 3    b  two  2.233232  0.249677}

In [51]:
pieces['a']

,key1,key2,data1,data2
0,a,one,1.483659,0.242712
1,a,two,0.091543,0.406203
4,a,one,0.966602,0.025725


In [52]:
pieces['b']

,key1,key2,data1,data2
2,b,one,2.543224,-0.714365
3,b,two,2.233232,0.249677


In [53]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [54]:
grouped = df.groupby(df.dtypes, axis=1)
# grouping by its type

In [56]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.483659  0.242712
1  0.091543  0.406203
2  2.543224 -0.714365
3  2.233232  0.249677
4  0.966602  0.025725
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [63]:
grouped_bytype = dict(list(df.groupby(df.dtypes, axis=1)))

In [74]:
list(grouped_bytype.keys())[1]

dtype('O')

In [66]:
grouped_bytype[np.dtype('float64')]

,data1,data2
0,1.483659,0.242712
1,0.091543,0.406203
2,2.543224,-0.714365
3,2.233232,0.249677
4,0.966602,0.025725


In [57]:
df

,key1,key2,data1,data2
0,a,one,1.483659,0.242712
1,a,two,0.091543,0.406203
2,b,one,2.543224,-0.714365
3,b,two,2.233232,0.249677
4,a,one,0.966602,0.025725


In [ ]:
df

---

## Selecting a Column or Subset of Columns

In [82]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.134218
      two     0.406203
b     one    -0.714365
      two     0.249677
Name: data2, dtype: float64

In [81]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.134218
     two   0.406203
b    one  -0.714365
     two   0.249677

In [83]:
pd.DataFrame(df.groupby(['key1', 'key2'])['data2'].mean())

data2
key1 key2          
a    one   0.134218
     two   0.406203
b    one  -0.714365
     two   0.249677

In [84]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [86]:
s_grouped.mean()

key1  key2
a     one     0.134218
      two     0.406203
b     one    -0.714365
      two     0.249677
Name: data2, dtype: float64

----

## Grouping with Dicts and Series

In [87]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [88]:
people.iloc[2:3, [1, 2]] = np.nan

In [89]:
people

,a,b,c,d,e
Joe,-0.931085,-0.307022,0.048735,0.497468,-0.557919
Steve,-1.433171,-0.788875,1.318014,-0.375390,1.356824
Wes,1.061945,NaN,NaN,-1.440461,-1.026906
Jim,-1.559543,0.219727,0.391310,-0.936556,-0.900283
Travis,0.587102,-0.311731,-0.619111,-0.291847,-0.084297


In [91]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [92]:
by_column = people.groupby(mapping, axis=1)

In [93]:
by_column.sum()

,blue,red
Joe,0.546203,-1.796027
Steve,0.942624,-0.865222
Wes,-1.440461,0.035039
Jim,-0.545247,-2.240100
Travis,-0.910958,0.191075


In [94]:
map_series = pd.Series(mapping)

In [95]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [98]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


---

## Grouping with Functions

In [99]:
people

,a,b,c,d,e
Joe,-0.931085,-0.307022,0.048735,0.497468,-0.557919
Steve,-1.433171,-0.788875,1.318014,-0.375390,1.356824
Wes,1.061945,NaN,NaN,-1.440461,-1.026906
Jim,-1.559543,0.219727,0.391310,-0.936556,-0.900283
Travis,0.587102,-0.311731,-0.619111,-0.291847,-0.084297


In [100]:
people.groupby(len)

In [101]:
for key, group in people.groupby(len):
    print(key)
    print(group); print()
# grouping based on string length
# using index

3
            a         b         c         d         e
Joe -0.931085 -0.307022  0.048735  0.497468 -0.557919
Wes  1.061945       NaN       NaN -1.440461 -1.026906
Jim -1.559543  0.219727  0.391310 -0.936556 -0.900283

5
              a         b         c        d         e
Steve -1.433171 -0.788875  1.318014 -0.37539  1.356824

6
               a         b         c         d         e
Travis  0.587102 -0.311731 -0.619111 -0.291847 -0.084297



In [102]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.428683,-0.087296,0.440044,-1.879549,-2.485108
5,-1.433171,-0.788875,1.318014,-0.375390,1.356824
6,0.587102,-0.311731,-0.619111,-0.291847,-0.084297


In [103]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [104]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.931085 -0.307022  0.048735 -1.440461 -1.026906
  two -1.559543  0.219727  0.391310 -0.936556 -0.900283
5 one -1.433171 -0.788875  1.318014 -0.375390  1.356824
6 two  0.587102 -0.311731 -0.619111 -0.291847 -0.084297

---

## Grouping by Index Levels

In [105]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [106]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           codes=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [107]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.028489 -0.746590 -1.033532 -0.021564 -1.514523
1      0.905257 -1.157990 -0.174296 -1.112140  2.242476
2      0.775164 -1.077799  0.436171 -0.300137 -0.131447
3      0.114578  1.010976  1.160737  0.754378 -0.181270

In [108]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
